In [ ]:
#default_exp synthesizer

# Synthesizer
> Loading, defining and training wavenet based speech synthesizers from various audio representations.

# Imports

In [ ]:
from fastai2.basics import *
from fastai2.callback.all import *

import librosa
import librosa.display
import IPython

In [ ]:
file = '/home/cerberus/.fastai/data/es/clips/'

# DataLoaders

In [ ]:
cqt = partial(librosa.cqt, n_bins=80)

In [ ]:
data = DataBlock((SpecBlock(cqt), SignalBlock(44100)),
                 get_items=get_audio_files,
                 splitter=RandomSplitter())

dls = data.dataloaders(file, batch_tfms=ToTensor(), bs=1)

# Loading Model

In [ ]:
model = torch.hub.load('nvidia/DeepLearningExamples:torchhub', 'nvidia_waveglow').cuda()

Using cache found in /home/cerberus/.cache/torch/hub/nvidia_DeepLearningExamples_torchhub


# Training

In [ ]:
learner = Learner(dls, model, loss_func=L1LossFlat())

In [ ]:
class JoinCB(Callback):
    def begin_batch(self):
        self.learn.xb = ((self.learn.xb, self.learn.yb),)

In [ ]:
learner.fit(1, lr=0, cbs=JoinCB())

start spect torch.Size([1, 80, 319])
start audio torch.Size([1, 325988])
upsample spect torch.Size([1, 80, 82432])


AssertionError: 

In [ ]:
learner.fit_one_cycle(4)

In [ ]:
b = learner.dls.one_batch()
learner.one_batch(1,b)

AttributeError: 'Learner' object has no attribute 'pred'

In [ ]:
learner.predict()

AttributeError: 'Learner' object has no attribute 'pred'

# Results